In [2]:
from utils_scrape import *
import os
import pandas as pd
import numpy as np
from collections import defaultdict
import datetime

In [3]:
%load_ext autoreload
%autoreload 2

In [19]:
%run -i keys.py

### Match Data - Singles

In [21]:
singles=[i for i in match_links if 'doub' not in i]
singles=[i for i in singles if ('futures' not in i) and ('amateur' not in i)]

In [6]:
def parse(t):
    string_ = str(t)
    try:
        return datetime.date(int(string_[:4]), int(string_[4:6]), int(string_[6:]))
    except:
        print ("Error",len(string_),string_)
        return datetime.date(1900,1,1)
    
#date_parser is depreciated in pd v2.0.0, making this function redundant

In [7]:
dtypes={'tourney_id': 'str',
        'draw_size': 'str',
        'winner_id': 'Int64',
        'loser_id': 'Int64',
       }

In [29]:
df=fetch_data(singles,
              "https://raw.githubusercontent.com",
              dtype=dtypes,
              date_format='%Y%m%d',
              parse_dates=['tourney_date'])

In [31]:
#Manual adjustments (necessarily as one can't determine these without looking at other databases)
df.loc[(df.tourney_id=='1963-580')&(df.match_num==22),'score']="6-3 6-4 4-6 6-3" 
df.loc[(df.tourney_id=='2001-M-SA-ITA-03A-2001a')&(df.match_num==9),'score']="6-3 6-3" 
df.loc[(df.tourney_id=='1918-560')&(df.match_num==14),'score']="6-4 6-0 6-3"

In [32]:
df['retirement']=df.score.apply(ret_parser)
df.loc[:,'score']=df.score.apply(sc_clean)
df=df.sort_values(by=['tourney_date','tourney_id','match_num'])

In [33]:
games=pd.DataFrame(games_ext(df.score),
                   columns=['W'+str(i+1) for i in range(5)]+['L'+str(i+1) for i in range(5)])
df=pd.concat([df,games],axis=1)

In [34]:
relabel={'G': 'Grand Slam',
         'D': 'Davis Cup',
         'M': 'Masters',
         'C': 'Challenger',
         'S': 'Futures',         
}
df['tourney_level']=df.tourney_level.replace(relabel)

In [35]:
df.to_csv('csvs/atp_matches.csv', index=False)

## Reading

In [235]:
dtypes={'tourney_id': str,
        'draw_size': str,
        'winner_id': 'Int64',
        'loser_id': 'Int64',
        'winner_seed':  str,
        'winner_entry': str,
        'loser_entry': str,
        'loser_seed':  str,
       }

dtypes={**dtypes,**{'W'+str(i+1):'Int64' for i in range(5)},**{'L'+str(i+1):'Int64' for i in range(5)}}

In [236]:
dfr=pd.read_csv('csvs/atp_matches.csv',dtype=dtypes, date_format='%Y%m%d',parse_dates=['tourney_date'])

## Cleaning